Importing Bibliothiques

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from string import punctuation
from os import listdir
from pickle import dump
from sklearn.utils import shuffle
import string
import joblib
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
# import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def load_data(filename):
    file=open(filename,'r')
    text=file.read()
    file.close()
    return text
def extract_keywords(text):
    tokens = text.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [w.lower() for w in tokens if w.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [w for w in tokens if len(w) > 1]
    return ' '.join(tokens)
def Load_All_data(directory):
    data=list()
    for filename in listdir(directory):
        path=directory+'/'+filename

        text=extract_keywords(load_data(path))
        data.append(text)
    return data


Load Data and preprocessing

In [5]:
# Load and preprocess data
neg_data = Load_All_data('./data/neg/')
pos_data = Load_All_data('./data/pos/')

# Combine negative and positive data
all_data = neg_data + pos_data
all_labels = [0] * len(neg_data) + [1] * len(pos_data)

# Shuffle the data and labels together
shuffled_data, shuffled_labels = shuffle(all_data, all_labels, random_state=42)

# Split data into training and testing sets tarin 90%   test 10%
train_data,y_train,test_data,y_test = shuffled_data[:900],shuffled_labels[:900],shuffled_data[900:],shuffled_labels[900:]

# Convert text data to TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(train_data)
X_test = tfidf_vectorizer.transform(test_data)
data_filenames = ['X_train.pkl', 'y_train.pkl', 'X_test.pkl', 'y_test.pkl']

for filename, data in zip(data_filenames, [X_train, y_train, X_test, y_test]):
    joblib.dump(data, filename)

create and train the model 

In [4]:
# Train a Naive Bayes classifier
model = SVC()
model.fit(X_train, y_train)
# Make predictions on the test data
y_pred = model.predict(X_test)
# Evaluate the model
print(classification_report(y_test, y_pred))
y_pred2 = model.predict(X_train)
# Evaluate the model
print(classification_report(y_train, y_pred2))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       540
           1       0.82      0.81      0.81       560

    accuracy                           0.81      1100
   macro avg       0.81      0.81      0.81      1100
weighted avg       0.81      0.81      0.81      1100

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       460
           1       1.00      1.00      1.00       440

    accuracy                           1.00       900
   macro avg       1.00      1.00      1.00       900
weighted avg       1.00      1.00      1.00       900



predict

In [5]:
def predict_review(review): 
    preprocessed_review = extract_keywords(review)  
    # Convertir l'avis prétraité en caractéristiques TF-IDF
    tfidf_vector = tfidf_vectorizer.transform([preprocessed_review])
    # Faire une prédiction avec le modèle
    predicted_sentiment = model.predict(tfidf_vector)
    # Interpréter la prédiction
    if predicted_sentiment[0] == 0:
        sentiment_label = "Négatif"
    else:
        sentiment_label = "Positif"
    print(f"Votre avis est prédit comme : {sentiment_label}")


In [8]:
predict_review("it'is amazing")

Votre avis est prédit comme : Positif
